In [1]:
import torch
from slowfast.utils.parser import load_config, parse_args
from slowfast.config.defaults import get_cfg
from pprint import pprint

In [38]:
cfg = get_cfg()
cfg.merge_from_file('./configs/Kinetics/pytorchvideo/X3D_M.yaml')

### Try to load from torchhub (doesnt work)

The model weights in pyslowfast has slightly difference with those in PyTorchVideo official model zoo. The layer names of above model weights will contain the additional prefix of `model`. due to the model wrapper in PySlowFast. https://github.com/facebookresearch/SlowFast/blob/main/projects/pytorchvideo/README.md

In [46]:
from collections import OrderedDict

old_chkpt = torch.load('checkpoints/ptvX3D-m-00270.pyth')

def remove_prefix(prev_odict):
    new_odict = OrderedDict()
    for key, value in prev_odict.items():
        new_key = key.split('model.')[1]
        new_odict[new_key] = value
    return new_odict

new_chkpt = {}

for key, value in old_chkpt.items():
    print(key, type(value))
    if key == 'model_state':  
        new_chkpt[key] = remove_prefix(value)
    else:
        new_chkpt[key] = value
    
torch.save(new_chkpt, "./checkpoints/renamed-ptvX3D-m-00270.pyth")

epoch <class 'int'>
model_state <class 'collections.OrderedDict'>
optimizer_state <class 'dict'>
cfg <class 'str'>


In [59]:
chkpt = torch.load('checkpoints/renamed-ptvX3D-m-00270.pyth')
chkpt['epoch']
# chkpt

269

In [56]:
model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_m', pretrained=True)
print(model)
# chkpt = torch.load('checkpoints/renamed-ptvX3D-m-00270.pyth')
# model.load_state_dict(chkpt)
# model.eval()


Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv2plus1d(
        (conv_t): Conv3d(3, 24, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
        (conv_xy): Conv3d(24, 24, kernel_size=(5, 1, 1), stride=(1, 1, 1), padding=(2, 0, 0), groups=24, bias=False)
      )
      (norm): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(24, 24, kernel_size=(1, 1, 1), stride=(1, 2, 2), bias=False)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(24, 54, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_a): BatchNorm3d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(54, 54, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), groups=54, bias=False)
            (nor

Using cache found in /home/sibi/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [5]:
# from build.lib.slowfast.models.ptv_model_builder import PTVX3D

# model = PTVX3D(cfg)
# model.load_state_dict(torch.load('checkpoints/ptvX3D-m-00270.pyth'))
# model.eval()

### Build model

In [5]:
from slowfast.models import build_model
import slowfast.utils.checkpoint as cu

print('Building model')
model = build_model(cfg, gpu_id=torch.cuda.current_device())
model.eval()

# print(model)

print('Start loading weights')
cu.load_test_checkpoint(cfg, model)
print('Done loading weights')

device = "cuda"
model = model.to(device)

Building model
Start loading weights
Done loading weights


In [64]:
pretrained = torch.load('checkpoints/X3D_M_pretrained.pyth')
pretrained.keys() # dict_keys(['epoch', 'model_state', 'optimizer_state', 'cfg'])

pprint(pretrained['cfg'])

('AVA:\n'
 '  ANNOTATION_DIR: '
 '/mnt/vol/gfsai-flash3-east/ai-group/users/haoqifan/ava/frame_list/\n'
 '  BGR: false\n'
 '  DETECTION_SCORE_THRESH: 0.9\n'
 '  EXCLUSION_FILE: ava_val_excluded_timestamps_v2.2.csv\n'
 '  FRAME_DIR: /mnt/fair-flash3-east/ava_trainval_frames.img/\n'
 '  FRAME_LIST_DIR: '
 '/mnt/vol/gfsai-flash3-east/ai-group/users/haoqifan/ava/frame_list/\n'
 '  FULL_TEST_ON_VAL: false\n'
 '  GROUNDTRUTH_FILE: ava_val_v2.2.csv\n'
 '  IMG_PROC_BACKEND: cv2\n'
 '  LABEL_MAP_FILE: ava_action_list_v2.2_for_activitynet_2019.pbtxt\n'
 '  TEST_FORCE_FLIP: false\n'
 '  TEST_LISTS:\n'
 '  - val.csv\n'
 '  TEST_PREDICT_BOX_LISTS:\n'
 '  - ava_val_predicted_boxes.csv\n'
 '  TRAIN_GT_BOX_LISTS:\n'
 '  - ava_train_v2.2.csv\n'
 '  TRAIN_LISTS:\n'
 '  - train.csv\n'
 '  TRAIN_PCA_EIGVAL:\n'
 '  - 0.225\n'
 '  - 0.224\n'
 '  - 0.229\n'
 '  TRAIN_PCA_EIGVEC:\n'
 '  - - -0.5675\n'
 '    - 0.7192\n'
 '    - 0.4009\n'
 '  - - -0.5808\n'
 '    - -0.0045\n'
 '    - -0.814\n'
 '  - - -0.5836\n

### Load classnames

In [6]:
# load classnames
import os
with open(os.path.join(cfg.DATA.PATH_TO_DATA_DIR, "classnames.txt")) as f:
    classnames = list(map(lambda l : l.strip(), f.readlines()))
classnames

['arrest_nut_and_dust_cup',
 'torque_tool_1_tight',
 'electric_coupler',
 'qr_code_scanning',
 'lock_nut_and_washer',
 'torque_tool_2_tight']

### Inference

In [8]:
from pytorchvideo.data.encoded_video import EncodedVideo

from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)

mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
frames_per_second = cfg.DATA.TARGET_FPS

transform_params = {
    "side_size": 256,
    "crop_size": cfg.DATA.TEST_CROP_SIZE,     # 256
    "num_frames": cfg.DATA.NUM_FRAMES,        # 16
    "sampling_rate": cfg.DATA.SAMPLING_RATE,  # 5
}

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(transform_params["num_frames"]),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=transform_params["side_size"]),
            # CenterCropVideo(
            #     crop_size=(transform_params["crop_size"], transform_params["crop_size"])
            # )
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (transform_params["num_frames"] * transform_params["sampling_rate"])/frames_per_second
print(f'{clip_duration = }')

clip_duration = 3.2


In [70]:
video_path = "../action_clips_resized/lock_nut_and_washer/003.mp4"
video = EncodedVideo.from_path(video_path)

start_sec = 0
end_sec = start_sec + clip_duration

# Load the desired clip
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

# Apply a transform to normalize the video input
video_data = transform(video_data)

# send to device
inputs = [video_data["video"].unsqueeze(0)]
for i in range(len(inputs)):
    inputs[i] = inputs[i].cuda(device=torch.device(torch.cuda.current_device()), non_blocking=True)

In [ ]:
import pandas as pd
import numpy as np

test_files = pd.read_csv(os.path.join(cfg.DATA.PATH_TO_DATA_DIR, "test.csv")).to_numpy()
test_files = list(map(lambda line: (line[0].split()), test_files))
test_files = np.array(test_files)

filenames, true_labels = test_files[:, 0], test_files[:, 1].astype('int')
filenames, true_labels

In [67]:
inputs = None

for file in filenames:
    video_path = os.path.join(cfg.DATA.PATH_PREFIX, file)
    video = EncodedVideo.from_path(video_path)

    start_sec = 0
    end_sec = start_sec + clip_duration

    # Load the desired clip
    video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

    # Apply a transform to normalize the video input
    video_data = transform(video_data)
    
    if inputs is None:
        # inputs = torch.tensor((len(filenames), 3, 16, 512, 256), dtype=video_data["video"].dtype)
        print(f'video data: {video_data["video"].shape}, dtype: {video_data["video"].dtype}')
        inputs = video_data["video"].unsqueeze(0)
        
    else:
        inputs = torch.cat((inputs, video_data["video"].unsqueeze(0)), dim=0)
        
    print('inputs: ', inputs.shape)


video data: torch.Size([3, 16, 512, 256]), dtype: torch.float32
inputs:  torch.Size([1, 3, 16, 512, 256])
inputs:  torch.Size([2, 3, 16, 512, 256])
inputs:  torch.Size([3, 3, 16, 512, 256])
inputs:  torch.Size([4, 3, 16, 512, 256])
inputs:  torch.Size([5, 3, 16, 512, 256])
inputs:  torch.Size([6, 3, 16, 512, 256])
inputs:  torch.Size([7, 3, 16, 512, 256])
inputs:  torch.Size([8, 3, 16, 512, 256])
inputs:  torch.Size([9, 3, 16, 512, 256])
inputs:  torch.Size([10, 3, 16, 512, 256])
inputs:  torch.Size([11, 3, 16, 512, 256])
inputs:  torch.Size([12, 3, 16, 512, 256])
inputs:  torch.Size([13, 3, 16, 512, 256])
inputs:  torch.Size([14, 3, 16, 512, 256])


(torch.Size([14, 3, 16, 512, 256]), device(type='cpu'))

In [60]:
inputs = [inputs]
# send to device
for i in range(len(inputs)):
    inputs[i] = inputs[i].cuda(device=torch.device(torch.cuda.current_device()), non_blocking=True)

In [62]:
inputs[0].shape, inputs[0].device

(torch.Size([14, 3, 16, 512, 256]), device(type='cuda', index=0))

In [69]:
inputs = [inputs]
model.cpu()

with torch.no_grad():
    preds = model(inputs)
    print(preds)
    idx = preds.argmax().item()
    print(classnames[idx])

tensor([[9.4740e-01, 3.0093e-02, 1.8047e-04, 4.9500e-04, 2.6498e-03, 1.9182e-02],
        [9.8813e-01, 6.3056e-03, 8.1251e-04, 5.4157e-04, 3.9090e-03, 3.0257e-04],
        [7.6553e-03, 9.6233e-01, 2.4272e-03, 1.4813e-02, 1.0524e-02, 2.2531e-03],
        [2.0256e-04, 9.9930e-01, 3.9909e-07, 7.7910e-05, 3.6576e-05, 3.8208e-04],
        [2.3817e-05, 1.2864e-05, 8.4586e-01, 1.2863e-02, 1.4123e-01, 7.0756e-06],
        [5.1262e-04, 2.3764e-04, 7.3574e-01, 1.5362e-01, 1.0980e-01, 9.2810e-05],
        [4.1621e-04, 2.9776e-04, 8.2684e-01, 8.0071e-03, 1.6432e-01, 1.2465e-04],
        [8.6852e-05, 6.5848e-05, 1.2116e-01, 7.6908e-01, 1.0951e-01, 1.0030e-04],
        [9.7614e-06, 3.6615e-06, 1.1909e-02, 9.7819e-01, 4.2803e-03, 5.6078e-03],
        [7.5789e-05, 1.3547e-04, 6.5626e-01, 2.3917e-01, 1.0421e-01, 1.4119e-04],
        [2.1548e-01, 1.0911e-03, 3.1870e-02, 1.8255e-03, 7.4354e-01, 6.1953e-03],
        [6.9688e-01, 8.7033e-05, 6.9955e-02, 5.3238e-04, 2.3237e-01, 1.7838e-04],
        [8.6112e

IndexError: list index out of range

In [91]:
preds_max_idx = preds.argmax(dim=1).numpy()
preds_max_idx, true_labels

(array([0, 0, 1, 1, 2, 2, 2, 3, 3, 2, 4, 0, 5, 5]),
 array([0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5]))

In [96]:
for pred, true in zip(preds_max_idx, true_labels):
    if pred != true:
        print(f'true: {classnames[true]} \t\t pred: {classnames[pred]}')

true: qr_code_scanning 		 pred: electric_coupler
true: lock_nut_and_washer 		 pred: arrest_nut_and_dust_cup


In [64]:
torch.cuda.empty_cache()